In [1]:
%load_ext autoreload
%autoreload 2
import os, sys, re, datetime, random, gzip, json, copy
from pathlib import Path
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import collections
import networkx as nx
import numpy as np
import pandas as pd
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
PROJ_PATH = Path(os.path.join(re.sub("/CTGCN.*$", '', os.getcwd()), 'CTGCN'))
sys.path.insert(1, str(str(PROJ_PATH.parents[0] / 'DySubG/src/')))
from ranking import Evaluation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score

# Utils

In [29]:
def get_training_data(data, node_embedding, all_labels, label_mapping):
    train_pos = []
    train_neg = []
    val_pos = []
    val_neg = []
    test_pos = []
    test_neg = []
    for i,d in data.items():
        node_id = d['node_id']
        time_id = d['time_id']
        pos_labels = d['label']
        neg_labels = [i for i in all_labels if i not in pos_labels]
        if len(pos_labels) > 0:
            for l in pos_labels:
                label_nid = label_mapping[l]
                if node_id in node_embedding[time_id] and label_nid in node_embedding[time_id]:
                    pos_pair = (
                        np.array(node_embedding[time_id][node_id]), 
                        np.array(node_embedding[time_id][label_nid]))
                    if d['dataset'] == 'train':
                        train_pos.append(pos_pair)
                    elif d['dataset'] == 'val':
                        val_pos.append(pos_pair)
                    elif d['dataset'] == 'test':
                        test_pos.append(pos_pair)
        if len(neg_labels) > 0:   
            for l in neg_labels:
                label_nid = label_mapping[l]
                if node_id in node_embedding[time_id] and label_nid in node_embedding[time_id]:
                    neg_pair = (
                        np.array(node_embedding[time_id][node_id]), 
                        np.array(node_embedding[time_id][label_nid]))
                    if d['dataset'] == 'train':
                        train_neg.append(neg_pair)
                    elif d['dataset'] == 'val':
                        val_neg.append(neg_pair)
                    elif d['dataset'] == 'test':
                        test_neg.append(neg_pair)
    return train_pos, train_neg, val_pos, val_neg, test_pos, test_neg


def get_link_score(fu, fv, operator='HAD'):
    """Given a pair of embeddings, compute link feature based on operator (such as Hadammad product, etc.)"""
    fu = np.array(fu)
    fv = np.array(fv)
    if operator == 'HAD':
        return np.multiply(fu, fv)
    elif operator == 'AVG':
        return (fu + fv) / 2
    elif operator == 'L1':
        return np.abs(fu - fv)
    elif operator == 'L2':
        return (fu - fv) ** 2
    else:
        raise NotImplementedError

def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
def predict_link_without_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg):
    pred_train = [sigmoid(np.dot(e[0], e[1].T)) for e in train_pos + train_neg]
    label_train = [1] * len(train_pos) + [0] * len(train_neg)
    pred_val = [sigmoid(np.dot(e[0], e[1].T)) for e in val_pos + val_neg]
    label_val = [1] * len(val_pos) + [0] * len(val_neg)
    pred_test = [sigmoid(np.dot(e[0], e[1].T)) for e in test_pos + test_neg]
    label_test = [1] * len(test_pos) + [0] * len(test_neg)
    return pred_train, label_train, pred_val, label_val, pred_test, label_test


def predict_link_with_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operator):
    train_feats = np.array([get_link_score(e[0], e[1], operator) for e in train_pos + train_neg])
    val_feats = np.array([get_link_score(e[0], e[1], operator) for e in val_pos + val_neg])
    test_feats = np.array([get_link_score(e[0], e[1], operator) for e in test_pos + test_neg])
    label_train = np.array([1] * len(train_pos) + [0] * len(train_neg))
    label_val = np.array([1] * len(val_pos) + [0] * len(val_neg))
    label_test = np.array([1] * len(test_pos) + [0] * len(test_neg))
    
    clf = linear_model.LogisticRegression(max_iter=5000)
    clf.fit(train_feats, label_train)
    pred_train = clf.predict_proba(train_feats)[:, 1]
    pred_val = clf.predict_proba(val_feats)[:, 1]
    pred_test = clf.predict_proba(test_feats)[:, 1]

    return pred_train, label_train, pred_val, label_val, pred_test, label_test, clf

def calc_auc(label_train, pred_train):
    auc = roc_auc_score(label_train, pred_train)
    if auc < 0.5:
        return 1 - auc
    else: 
        return auc
    
def evaluate_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operators=['HAD'], threshold=0.5):
    results = {}
    models = {}
    # Predict without classifier
    pred_train, label_train, pred_val, label_val, pred_test, label_test = predict_link_without_classifier(
        train_pos, train_neg, val_pos, val_neg, test_pos, test_neg)
    results['sigmoid_auc'] = {
        'train': calc_auc(label_train, pred_train), 
        'val': calc_auc(label_val, pred_val),
        'test': calc_auc(label_test, pred_test), 
    }
    results['sigmoid_f1']  = {
        'train': f1_score(label_train, [1 if i >= threshold else 0 for i in pred_train]), 
        'val': f1_score(label_val, [1 if i >= threshold else 0 for i in pred_val]),
        'test': f1_score(label_test,[1 if i >= threshold else 0 for i in pred_test]),
    }
    
    # Predict with classifier
    for operator in operators:
        pred_train, label_train, pred_val, label_val, pred_test, label_test, clf = predict_link_with_classifier(
            train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operator)
        results[f'{operator}_auc'] = {
            'train': calc_auc(label_train, pred_train), 
            'val': calc_auc(label_val, pred_val),
            'test': calc_auc(label_test, pred_test), 
        }
        results[f'{operator}_f1']  = {
            'train': f1_score(label_train, [1 if i >= threshold else 0 for i in pred_train]), 
            'val': f1_score(label_val, [1 if i >= threshold else 0 for i in pred_val]),
            'test': f1_score(label_test,[1 if i >= threshold else 0 for i in pred_test]),
        }
        models[operator] = clf
    return results, models

def eval_lp(data, all_labels, label_mapping, method='CTGCN-C', num_time_steps=5, exp='imdb', operators=['HAD'], verbose=True, threshold=0.5):
    if method in ['DynAE', 'DynRNN', 'DynAERNN', 'DHNE']:
        data = {i:j for i,j in data.items() if j['time_id']>=1}
    nodes = pd.read_csv(f'./data/{exp}/nodes_set/nodes.csv', names=['nodes'])['nodes'].values
    start_idx = min([d['time_id'] for i,d in data.items()])
    node_embedding = {}
    for time_id in range(start_idx, num_time_steps):
        if method == 'DyHATR':
            if exp == 'dblp':
                if time_id == 4:
                    node_embedding[4] = {}
                else:
                    embs = pd.read_csv(
                    './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t')
                    dict_embs = embs.T.to_dict('list')
                    node_embedding[time_id] = dict_embs
            else:
                embs = pd.read_csv(
                    './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t')
                dict_embs = embs.T.to_dict('list')
                node_embedding[time_id] = dict_embs
        else:
            embs = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values
            dict_embs = {k:v for k,v in zip(nodes, embs)}
            node_embedding[time_id] = dict_embs
           
    train_pos, train_neg, val_pos, val_neg, test_pos, test_neg = get_training_data(
        data, node_embedding, all_labels, label_mapping)
    if verbose:
        print(len(train_pos), len(train_neg), len(val_pos), len(val_neg), len(test_pos), len(test_neg))
    results = evaluate_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operators, threshold)
    return results


def print_report(exp='imdb', methods=[], selected_methods=[], threshold=0.5):
    if len(methods) == 0:
        methods = [
            'GCN', 'GAT', 'SAGE', 'GIN', 
            'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
            'GCRN', 'TIMERS', 'DynAE', 'DynRNN', 'DynAERNN', 'DynGEM', 'DySAT',
            'VGRNN', 'EvolveGCN', 'CTGCN-C', 'DHNE', 'DyHATR',
        ]
    if len(selected_methods) == 0:
        selected_methods = [
            'GCN', 'TgGAT', 'TgSAGE', 'TgGIN',
            'GCRN', 'TIMERS', 'DynAE', 'DynRNN', 'DynAERNN', 'DynGEM', 'DySAT',
            'VGRNN', 'EvolveGCN', 'CTGCN-C', 'DHNE', 'DyHATR',
        ]
        
    try:
        pd_edges = pd.read_csv(f'./data/{exp}/0.input/temporal_edge_list.txt', sep=' ', names=['source_id', 'target_id', 'time_id'])
    except:
        pd_edges = pd.read_csv(f'./data/{exp}/0.input/edges.csv')
    
    num_time_steps = pd_edges['time_id'].max() + 1
    if exp.startswith('imdb'):
        data = pd.read_pickle(f'./data/{exp}/0.input/data.pkl')
        all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
        label_mapping = pd.read_pickle(f'./data/{exp}/0.input/entity_mapping.pkl')['genre']
    elif exp.startswith('dblp'):
        data = pd.read_pickle(f'./data/{exp}/0.input/data.pkl')
        all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
        cid2cname = pd.read_pickle('../DySubG/dataset/dblp/cid2cname.pkl')
        cname2cid = {j:i for i,j in cid2cname.items() if j in all_labels}
        label_mapping = {j:i for i,j in cid2cname.items() if j in all_labels}
        
    operators = ['HAD', 'AVG', 'L1', 'L2']
    res = []
    model_ls = {}
    for method in methods:
        print(method)
        results, models = eval_lp(
            data, all_labels, label_mapping, method, num_time_steps, exp, operators, threshold=threshold)
        tmp = pd.DataFrame(results)
        tmp['method'] = method
        res.append(tmp)
        model_ls[method] = models
    
    df = pd.concat(res)
    df = df.reset_index().rename(columns={'index':'dataset'})
    df['best_AUC'] = df[['sigmoid_auc', 'HAD_auc', 'AVG_auc', 'L1_auc', 'L2_auc']].max(axis=1)
    df['best_F1'] = df[['sigmoid_f1', 'HAD_f1', 'AVG_f1', 'L1_f1', 'L2_f1']].max(axis=1)

    print('Full report')
    display(df)

    print('Test report')
    display(df[df['dataset']=='test'])

    print('Selected methods')
    display(df[(df['dataset']=='test')&(df['method'].isin(selected_methods))][[
        'method',
        'sigmoid_auc', 'HAD_auc', 'AVG_auc', 'L1_auc', 'L2_auc', 'best_AUC',
        'sigmoid_f1', 'HAD_f1', 'AVG_f1', 'L1_f1', 'L2_f1', 'best_F1']])
    
    return df, model_ls

def get_ground_truth(data, all_labels, label_mapping):
    ground_truth = {}
    pred_idx = {}
    for i,d in data.items():
        node_id = d['node_id']
        time_id = d['time_id']
        pos_labels = d['label']
        neg_labels = [i for i in all_labels if i not in pos_labels]
        if len(pos_labels) > 0:
            ground_truth[node_id] = [label_mapping[l] for l in pos_labels]
        pred_idx[node_id] = [label_mapping[l] for l in all_labels]
    return ground_truth, pred_idx

def make_prediction(pred_idx, node_embedding, models):
    ranking = {}
    distances = []
    for s, ts in pred_idx.items():
        for t in ts:
            if t in node_embedding and s in node_embedding:
                source = node_embedding[s]
                target = node_embedding[t]
                probs = sigmoid(np.dot(np.array(source), np.array(target).T))
                distances.append((s, t, probs))
        distances.sort(key=lambda tup: tup[2])
    ranking['sigmoid'] = distances  
    
    for operator, model in models.items():
        distances = []
        for s, ts in pred_idx.items():
            for t in ts:
                if t in node_embedding and s in node_embedding:
                    source = node_embedding[s]
                    target = node_embedding[t]
                    probs = sigmoid(np.dot(np.array(source), np.array(target).T))
                    distances.append((s, t, probs))
            distances.sort(key=lambda tup: tup[2])
        ranking[operator] = distances
    return ranking

# def eval_ranking(pred_dict, true_dict, k):
#     predicted_indices = [] # collections.OrderedDict()
#     true_indices = [] # collections.OrderedDict()
#     author_indices = sorted(list(true_dict.keys()))
#     for aid in author_indices:
#         predicted_indices.append(pred_dict[aid])
#         true_indices.append(true_dict[aid])
#     eval_agent = Evaluation(predicted_indices, true_indices, k)
#     return eval_agent.result

def calculate_multilabel(gt, prd):
    micro_f1 = f1_score(gt, prd, average='micro', zero_division=0)
    macro_f1 = f1_score(gt, prd, average='macro', zero_division=0)
    f1 = f1_score(gt, prd, average='weighted', zero_division=0)
    micro_recall = recall_score(gt, prd, average='micro', zero_division=0)
    macro_recall = recall_score(gt, prd, average='macro', zero_division=0)
    recall = recall_score(gt, prd, average='weighted', zero_division=0)
    micro_precision = precision_score(gt, prd, average='micro', zero_division=0)
    macro_precision = precision_score(gt, prd, average='macro', zero_division=0)
    precision = precision_score(gt, prd, average='weighted', zero_division=0)
    return f1, micro_f1, macro_f1, recall, micro_recall, macro_recall, precision, micro_precision, macro_precision

def evaluation(prd, gt, multilabel_binarizer, k=5):
    ground_truth = {}
    pred = prd
    for i, j in prd.items():
        ground_truth[i] = gt[i]
        
    gt = multilabel_binarizer.transform(list(ground_truth.values()))
    result = {}
    for k in range(1, k+1):
        prd = multilabel_binarizer.transform(list({i: j[:k] for i, j in pred.items()}.values()))
        result[k] = calculate_multilabel(gt, prd)
    df = pd.DataFrame(result).T
    df.columns = [
    'f1', 'micro_f1', 'macro_f1', 
    'recall', 'micro_recall', 'macro_recall', 
    'precision', 'micro_precision', 'macro_precision']
    df['k'] = range(1, k+1)
    return df

# Evaluate

## IMDB

In [31]:
embs = pd.read_csv(
                './data/imdb_lp_t2/2.embedding/DyHATR/00.csv', index_col=0, sep='\t')

In [41]:
methods = [
#     'GCN', 'GAT', #'SAGE', 'GIN', 
#     'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
#     'GCRN', 'TIMERS', 'DynAE', #'DynRNN', 
#     'DynAERNN', 'DynGEM', 'DySAT',
#     'VGRNN', 'EvolveGCN', 'CTGCN-C',
#     'DHNE', 
    'DyHATR',
]
df, model_ls = print_report(exp='imdb_lp_t2', methods=methods, threshold=0.1)

DyHATR
3021 3897 984 1326 954 1350
Full report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
0,train,0.533731,0.607908,0.544968,0.607908,0.574642,0.607908,0.583710,0.607908,0.555898,0.607908,DyHATR,0.583710,0.607908
1,val,0.522880,0.597450,0.525927,0.597450,0.548170,0.597450,0.563563,0.597450,0.541917,0.597450,DyHATR,0.563563,0.597450
2,test,0.508667,0.585635,0.521587,0.585635,0.523828,0.585635,0.519653,0.585635,0.518885,0.585635,DyHATR,0.523828,0.585635


Test report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
2,test,0.508667,0.585635,0.521587,0.585635,0.523828,0.585635,0.519653,0.585635,0.518885,0.585635,DyHATR,0.523828,0.585635


Selected methods


,method,sigmoid_auc,HAD_auc,AVG_auc,L1_auc,L2_auc,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,DyHATR,0.508667,0.521587,0.523828,0.519653,0.518885,0.523828,0.585635,0.585635,0.585635,0.585635,0.585635,0.585635


In [49]:
name = 'imdb_lp_t2'
time_id = 4 - 1
data = pd.read_pickle(f'./data/{name}/0.input/data.pkl')
all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
label_mapping = pd.read_pickle(f'./data/{name}/0.input/entity_mapping.pkl')['genre']
test_data = {i: d for i,d in data.items() if d['dataset']=='test'}
ground_truth, pred_idx = get_ground_truth(test_data, all_labels, label_mapping)
multilabel_binarizer = MultiLabelBinarizer().fit(list(ground_truth.values()))
cnames = [
    'method', 'k', 'operator',
    'f1', 'micro_f1', 'macro_f1', 
    'recall', 'micro_recall', 'macro_recall', 
    'precision', 'micro_precision', 'macro_precision']

In [50]:
ls = []
for method in methods:
    print(method)
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(
                    name, method, time_id), index_col=0, sep='\t').T.to_dict('list')
    models = model_ls[method]
    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = evaluation(pred, ground_truth, multilabel_binarizer, k=3)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    pd_res['method'] = method
    ls.append(pd_res)
results = pd.concat(ls)[cnames]

DyHATR


In [51]:
results

,method,k,operator,f1,micro_f1,macro_f1,recall,micro_recall,macro_recall,precision,micro_precision,macro_precision
1,DyHATR,1,sigmoid,0.310087,0.361208,0.311218,0.325996,0.325996,0.327315,0.442697,0.404948,0.442301
2,DyHATR,2,sigmoid,0.486837,0.490763,0.486789,0.640461,0.640461,0.640392,0.398133,0.397786,0.398076
3,DyHATR,3,sigmoid,0.585660,0.585635,0.585625,1.000000,1.000000,1.000000,0.414098,0.414062,0.414062
1,DyHATR,1,HAD,0.300234,0.343786,0.298904,0.310273,0.310273,0.308702,0.383590,0.385417,0.383636
2,DyHATR,2,HAD,0.479493,0.495582,0.480178,0.646751,0.646751,0.649030,0.398748,0.401693,0.398852
3,DyHATR,3,HAD,0.585660,0.585635,0.585625,1.000000,1.000000,1.000000,0.414098,0.414062,0.414062
1,DyHATR,1,AVG,0.196261,0.370499,0.195646,0.334382,0.334382,0.333333,0.138890,0.415365,0.138455
2,DyHATR,2,AVG,0.394625,0.514056,0.392156,0.670860,0.670860,0.666667,0.279528,0.416667,0.277778
3,DyHATR,3,AVG,0.585660,0.585635,0.585625,1.000000,1.000000,1.000000,0.414098,0.414062,0.414062
1,DyHATR,1,L1,0.237633,0.369338,0.236694,0.333333,0.333333,0.332165,0.483150,0.414062,0.484347


In [22]:
results.to_csv('./output/imdb.csv', index=False)

In [7]:
# k = 1
# operator = 'HAD'
# excl = ['TIMERS']
# incl = [
#     'GCN', 'TgGAT', 'TgSAGE', 'TgGIN', 'GCRN', 
#     'DynAE', 'DynAERNN', 'DynGEM', 'DySAT', 'VGRNN',
#     'EvolveGCN', 'CTGCN-C']
# display(results[(results['k']==k)&(results['operator']==operator)&(results['method'].isin(incl))])

## DBLP

In [9]:
methods = [
#     'GCN', 'GAT', #'SAGE', 'GIN', 
#     'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
#     'GCRN', 'TIMERS', 'DynAE',# 'DynRNN',
#     'DynAERNN', 'DynGEM', 'DySAT',
#     'VGRNN', 'EvolveGCN', 'CTGCN-C', 'DHNE',
    'DyHATR',
]
df, model_ls = print_report(exp='dblp', methods=methods, threshold=0.5)

DyHATR
3434 6636 1381 2719 757 1878
Full report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
0,train,0.528977,0.387724,0.559526,0.0,0.592082,0.045391,0.604294,0.079257,0.585332,0.004069,DyHATR,0.604294,0.387724
1,val,0.506957,0.348821,0.501546,0.0,0.531119,0.007210,0.540457,0.016973,0.526984,0.000000,DyHATR,0.540457,0.348821
2,test,0.521230,0.320732,0.511127,0.0,0.540177,0.000000,0.514841,0.005256,0.505739,0.000000,DyHATR,0.540177,0.320732


Test report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
2,test,0.52123,0.320732,0.511127,0.0,0.540177,0.0,0.514841,0.005256,0.505739,0.0,DyHATR,0.540177,0.320732


Selected methods


,method,sigmoid_auc,HAD_auc,AVG_auc,L1_auc,L2_auc,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,DyHATR,0.52123,0.511127,0.540177,0.514841,0.505739,0.540177,0.320732,0.0,0.0,0.005256,0.0,0.320732


In [30]:
name = 'dblp'
time_id = 7 - 1
data = pd.read_pickle(f'./data/{name}/0.input/data.pkl')
all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
cid2cname = pd.read_pickle('../DySubG/dataset/dblp/cid2cname.pkl')
cname2cid = {j:i for i,j in cid2cname.items() if j in all_labels}
label_mapping = {j:i for i,j in cid2cname.items() if j in all_labels}
test_data = {i: d for i,d in data.items() if d['dataset']=='test'}
ground_truth, pred_idx = get_ground_truth(test_data, all_labels, label_mapping)
# ground_truth = pd.read_pickle(f'./data/{name}/0.input/ground_truth.pkl')
print(len(all_labels))
cnames = [
    'method', 'k', 'operator',
    'f1', 'micro_f1', 'macro_f1', 
    'recall', 'micro_recall', 'macro_recall', 
    'precision', 'micro_precision', 'macro_precision']

5


In [31]:
ls = []
for method in methods:
    print(method)
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(
                    name, method, time_id), index_col=0, sep='\t').T.to_dict('list')
    models = model_ls[method]
    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    multilabel_binarizer = MultiLabelBinarizer().fit(list(ground_truth.values()))
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = evaluation(pred, ground_truth, multilabel_binarizer, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    pd_res['method'] = method
    ls.append(pd_res)
results = pd.concat(ls)[cnames]

DyHATR


In [32]:
results

,method,k,operator,f1,micro_f1,macro_f1,recall,micro_recall,macro_recall,precision,micro_precision,macro_precision
1,DyHATR,1,sigmoid,0.268923,0.286604,0.248085,0.243065,0.243065,0.222159,0.331486,0.349146,0.309351
2,DyHATR,2,sigmoid,0.373241,0.378796,0.349512,0.453104,0.453104,0.422458,0.319450,0.325427,0.300585
3,DyHATR,3,sigmoid,0.417909,0.414029,0.399227,0.639366,0.639366,0.620870,0.312942,0.306135,0.296952
4,DyHATR,4,sigmoid,0.437593,0.427923,0.421996,0.809775,0.809775,0.811962,0.305309,0.290797,0.290688
5,DyHATR,5,sigmoid,0.460797,0.446344,0.442192,1.000000,1.000000,1.000000,0.303212,0.287287,0.287287
1,DyHATR,1,HAD,0.268923,0.286604,0.248085,0.243065,0.243065,0.222159,0.331486,0.349146,0.309351
2,DyHATR,2,HAD,0.373241,0.378796,0.349512,0.453104,0.453104,0.422458,0.319450,0.325427,0.300585
3,DyHATR,3,HAD,0.417909,0.414029,0.399227,0.639366,0.639366,0.620870,0.312942,0.306135,0.296952
4,DyHATR,4,HAD,0.437593,0.427923,0.421996,0.809775,0.809775,0.811962,0.305309,0.290797,0.290688
5,DyHATR,5,HAD,0.460797,0.446344,0.442192,1.000000,1.000000,1.000000,0.303212,0.287287,0.287287


In [14]:
# results.to_csv('./output/dblp.csv', index=False)

In [6]:
k = 1
operator = 'HAD'
excl = ['TIMERS']
incl = [
    'GCN', 'TgGAT', 'TgSAGE', 'TgGIN', 'GCRN', 
    'DynAE', 'DynAERNN', 'DynGEM', 'DySAT', 'VGRNN',
    'EvolveGCN', 'CTGCN-C']
display(results[(results['k']==k)&(results['operator']==operator)&(results['method'].isin(incl))])

,method,k,operator,f1,micro_f1,macro_f1,recall,micro_recall,macro_recall,precision,micro_precision,macro_precision
1,GCN,1,HAD,0.159546,0.223602,0.157728,0.189723,0.189723,0.205900,0.281803,0.272212,0.258440
1,TgGAT,1,HAD,0.263816,0.271739,0.259002,0.230567,0.230567,0.229418,0.323808,0.330813,0.314775
1,TgSAGE,1,HAD,0.228076,0.231366,0.225328,0.196311,0.196311,0.200080,0.323405,0.281664,0.303094
1,TgGIN,1,HAD,0.097163,0.190994,0.115633,0.162055,0.162055,0.193575,0.193503,0.232514,0.194667
1,GCRN,1,HAD,0.285916,0.321429,0.265623,0.272727,0.272727,0.243929,0.381796,0.391304,0.370952
1,DynAE,1,HAD,0.057585,0.184783,0.073457,0.156785,0.156785,0.200000,0.035269,0.224953,0.044991
1,DynAERNN,1,HAD,0.064788,0.197205,0.077439,0.167325,0.167325,0.200000,0.040171,0.240076,0.048015
1,DynGEM,1,HAD,0.218643,0.260870,0.213713,0.221344,0.221344,0.223223,0.496377,0.317580,0.468767
1,DySAT,1,HAD,0.183850,0.214286,0.199190,0.181818,0.181818,0.205449,0.311793,0.260870,0.302988
1,VGRNN,1,HAD,0.142824,0.268634,0.131890,0.227931,0.227931,0.204912,0.185555,0.327032,0.200656


In [8]:
k = 2
operator = 'sigmoid'
excl = ['TIMERS']
incl = [
    'GCN', 'TgGAT', 'TgSAGE', 'TgGIN', 'GCRN', 
    'DynAE', 'DynAERNN', 'DynGEM', 'DySAT', 'VGRNN',
    'EvolveGCN', 'CTGCN-C']
display(results[(results['k']==k)&(results['operator']==operator)&(results['method'].isin(incl))])

,method,k,operator,f1,micro_f1,macro_f1,recall,micro_recall,macro_recall,precision,micro_precision,macro_precision
2,GCN,2,sigmoid,0.455404,0.445790,0.438294,0.533597,0.533597,0.528079,0.405459,0.382798,0.382426
2,TgGAT,2,sigmoid,0.372884,0.367639,0.363165,0.440053,0.440053,0.445920,0.342091,0.315690,0.321951
2,TgSAGE,2,sigmoid,0.334015,0.333517,0.316661,0.399209,0.399209,0.383889,0.293354,0.286389,0.276369
2,TgGIN,2,sigmoid,0.260246,0.308200,0.266808,0.368906,0.368906,0.413395,0.463730,0.264650,0.448451
2,GCRN,2,sigmoid,0.347472,0.365438,0.324554,0.437418,0.437418,0.414345,0.319586,0.313800,0.299698
2,DynAE,2,sigmoid,0.121453,0.269675,0.150403,0.322793,0.322793,0.400000,0.074810,0.231569,0.092628
2,DynAERNN,2,sigmoid,0.122372,0.270776,0.150896,0.324111,0.324111,0.400000,0.075440,0.232514,0.093006
2,DynGEM,2,sigmoid,0.311702,0.398459,0.275996,0.476943,0.476943,0.434567,0.332312,0.342155,0.309645
2,DySAT,2,sigmoid,0.351645,0.352229,0.335104,0.421607,0.421607,0.409211,0.311433,0.302457,0.294186
2,VGRNN,2,sigmoid,0.280326,0.416070,0.221335,0.498024,0.498024,0.392627,0.215061,0.357278,0.177874


## DBLP_v1

In [7]:
methods = [
    'GCN', #'GAT', #'SAGE', 'GIN', 
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'GCRN', 'TIMERS', #'DynAE',# 'DynRNN',
    'DynAERNN', 
    #'DynGEM', 'DySAT',
    'VGRNN', 'EvolveGCN', 'CTGCN-C',
]
name = 'dblp_1'
df, model_ls = print_report(exp=name, methods=methods, threshold=0.5)

GCN
5949 12311 1409 2786 759 1886
TgGCN
5949 12311 1409 2786 759 1886
TgGAT
5949 12311 1409 2786 759 1886
TgSAGE
5949 12311 1409 2786 759 1886
TgGIN
5949 12311 1409 2786 759 1886
GCRN
5949 12311 1409 2786 759 1886
TIMERS
5949 12311 1409 2786 759 1886
DynAERNN
5193 10667 1409 2786 759 1886
VGRNN
5949 12311 1409 2786 759 1886
EvolveGCN
5949 12311 1409 2786 759 1886
CTGCN-C
5949 12311 1409 2786 759 1886
Full report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
0,train,0.526001,0.491408,0.533809,0.000000,0.545078,0.000000,0.578678,0.000000,0.574706,0.000000,GCN,0.578678,0.491408
1,val,0.525065,0.502949,0.537274,0.000000,0.500011,0.000000,0.595616,0.000000,0.596057,0.000000,GCN,0.596057,0.502949
2,test,0.506734,0.445946,0.513291,0.000000,0.534008,0.000000,0.605521,0.000000,0.603805,0.000000,GCN,0.605521,0.445946
3,train,0.542268,0.366469,0.539496,0.000000,0.540818,0.000000,0.526681,0.000000,0.501842,0.000000,TgGCN,0.542268,0.366469
4,val,0.541877,0.277858,0.550526,0.000000,0.550719,0.000000,0.510368,0.000000,0.511940,0.000000,TgGCN,0.550719,0.277858
5,test,0.547636,0.285714,0.547475,0.000000,0.586429,0.000000,0.537434,0.000000,0.531616,0.000000,TgGCN,0.586429,0.285714
6,train,0.527311,0.491470,0.507841,0.000000,0.530537,0.000000,0.567958,0.000000,0.567586,0.000000,TgGAT,0.567958,0.491470
7,val,0.518090,0.502855,0.515226,0.000000,0.516180,0.000000,0.582513,0.000000,0.582459,0.000000,TgGAT,0.582513,0.502855
8,test,0.629691,0.445946,0.598588,0.000000,0.518447,0.000000,0.620653,0.000000,0.620919,0.000000,TgGAT,0.629691,0.445946
9,train,0.523992,0.483632,0.533024,0.000000,0.537363,0.000000,0.563708,0.000000,0.552757,0.000000,TgSAGE,0.563708,0.483632


Test report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
2,test,0.506734,0.445946,0.513291,0.000000,0.534008,0.000000,0.605521,0.000000,0.603805,0.000000,GCN,0.605521,0.445946
5,test,0.547636,0.285714,0.547475,0.000000,0.586429,0.000000,0.537434,0.000000,0.531616,0.000000,TgGCN,0.586429,0.285714
8,test,0.629691,0.445946,0.598588,0.000000,0.518447,0.000000,0.620653,0.000000,0.620919,0.000000,TgGAT,0.629691,0.445946
11,test,0.515838,0.447990,0.519817,0.000000,0.510762,0.000000,0.513604,0.000000,0.510152,0.000000,TgSAGE,0.519817,0.447990
14,test,0.545893,0.212598,0.595790,0.000000,0.541599,0.000000,0.571973,0.000000,0.566210,0.000000,TgGIN,0.595790,0.212598
17,test,0.510164,0.422525,0.529522,0.017880,0.531286,0.000000,0.545988,0.005202,0.543960,0.002618,GCRN,0.545988,0.422525
20,test,0.684041,0.509038,0.759308,0.370594,0.542276,0.000000,0.587028,0.330745,0.592923,0.330745,TIMERS,0.759308,0.509038
23,test,0.545828,0.445946,0.542423,0.000000,0.545950,0.000000,0.501427,0.000000,0.500615,0.000000,DynAERNN,0.545950,0.445946
26,test,0.579945,0.445946,0.578694,0.000000,0.604651,0.000000,0.550405,0.000000,0.555654,0.000000,VGRNN,0.604651,0.445946
29,test,0.518770,0.446578,0.598436,0.000000,0.612269,0.000000,0.524795,0.000000,0.524806,0.000000,EvolveGCN,0.612269,0.446578


Selected methods


,method,sigmoid_auc,HAD_auc,AVG_auc,L1_auc,L2_auc,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,GCN,0.506734,0.513291,0.534008,0.605521,0.603805,0.605521,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
8,TgGAT,0.629691,0.598588,0.518447,0.620653,0.620919,0.629691,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
11,TgSAGE,0.515838,0.519817,0.510762,0.513604,0.510152,0.519817,0.447990,0.000000,0.000000,0.000000,0.000000,0.447990
14,TgGIN,0.545893,0.595790,0.541599,0.571973,0.566210,0.595790,0.212598,0.000000,0.000000,0.000000,0.000000,0.212598
17,GCRN,0.510164,0.529522,0.531286,0.545988,0.543960,0.545988,0.422525,0.017880,0.000000,0.005202,0.002618,0.422525
20,TIMERS,0.684041,0.759308,0.542276,0.587028,0.592923,0.759308,0.509038,0.370594,0.000000,0.330745,0.330745,0.509038
23,DynAERNN,0.545828,0.542423,0.545950,0.501427,0.500615,0.545950,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
26,VGRNN,0.579945,0.578694,0.604651,0.550405,0.555654,0.604651,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
29,EvolveGCN,0.518770,0.598436,0.612269,0.524795,0.524806,0.612269,0.446578,0.000000,0.000000,0.000000,0.000000,0.446578
32,CTGCN-C,0.546413,0.548227,0.549895,0.547640,0.527792,0.549895,0.444901,0.138767,0.065854,0.332450,0.361538,0.444901


In [8]:
time_id = 7 - 1
data = pd.read_pickle(f'./data/{name}/0.input/data.pkl')
all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
cid2cname = pd.read_pickle('../DySubG/dataset/dblp/cid2cname.pkl')
cname2cid = {j:i for i,j in cid2cname.items() if j in all_labels}
label_mapping = {j:i for i,j in cid2cname.items() if j in all_labels}
test_data = {i: d for i,d in data.items() if d['dataset']=='test'}
ground_truth, pred_idx = get_ground_truth(test_data, all_labels, label_mapping)
# ground_truth = pd.read_pickle(f'./data/{name}/0.input/ground_truth.pkl')
print(len(all_labels))
cnames = [
    'method', 'k', 'operator',
    'f1', 'micro_f1', 'macro_f1', 
    'recall', 'micro_recall', 'macro_recall', 
    'precision', 'micro_precision', 'macro_precision']

5


In [9]:
ls = []
for method in methods:
    print(method)
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(
                    name, method, time_id), index_col=0, sep='\t').values
    models = model_ls[method]
    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    multilabel_binarizer = MultiLabelBinarizer().fit(list(ground_truth.values()))
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = evaluation(pred, ground_truth, multilabel_binarizer, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    pd_res['method'] = method
    ls.append(pd_res)
results = pd.concat(ls)[cnames]

GCN
TgGCN
TgGAT
TgSAGE
TgGIN
GCRN
TIMERS
DynAERNN
VGRNN
EvolveGCN
CTGCN-C


In [11]:
k = 1
operator = 'sigmoid'
excl = ['TIMERS']
incl = [
    'GCN', 'TgGAT', 'TgSAGE', 'TgGIN', 'GCRN', 
    'DynAE', 'DynAERNN', 'DynGEM', 'DySAT', 'VGRNN',
    'EvolveGCN', 'CTGCN-C']
display(results[(results['k']==k)&(results['operator']==operator)&(results['method'].isin(incl))])

,method,k,operator,f1,micro_f1,macro_f1,recall,micro_recall,macro_recall,precision,micro_precision,macro_precision
1,GCN,1,sigmoid,0.173650,0.276398,0.172525,0.234519,0.234519,0.219798,0.182171,0.336484,0.196038
1,TgGAT,1,sigmoid,0.394481,0.394410,0.384465,0.334651,0.334651,0.330313,0.497196,0.480151,0.476017
1,TgSAGE,1,sigmoid,0.192286,0.231366,0.171433,0.196311,0.196311,0.178623,0.216579,0.281664,0.195448
1,TgGIN,1,sigmoid,0.123789,0.214286,0.119196,0.181818,0.181818,0.199836,0.507012,0.260870,0.460500
1,GCRN,1,sigmoid,0.246601,0.290373,0.219265,0.246377,0.246377,0.208783,0.308716,0.353497,0.289728
1,DynAERNN,1,sigmoid,0.113483,0.270186,0.099004,0.229249,0.229249,0.200000,0.075405,0.328922,0.065784
1,VGRNN,1,sigmoid,0.161118,0.330745,0.114825,0.280632,0.280632,0.200000,0.112996,0.402647,0.080529
1,EvolveGCN,1,sigmoid,0.183122,0.217391,0.174016,0.184453,0.184453,0.179508,0.272406,0.264650,0.261822
1,CTGCN-C,1,sigmoid,0.221548,0.263975,0.194072,0.223979,0.223979,0.197355,0.286757,0.321361,0.260237


In [12]:
k = 2
operator = 'sigmoid'
excl = ['TIMERS']
incl = [
    'GCN', 'TgGAT', 'TgSAGE', 'TgGIN', 'GCRN', 
    'DynAE', 'DynAERNN', 'DynGEM', 'DySAT', 'VGRNN',
    'EvolveGCN', 'CTGCN-C']
display(results[(results['k']==k)&(results['operator']==operator)&(results['method'].isin(incl))])

,method,k,operator,f1,micro_f1,macro_f1,recall,micro_recall,macro_recall,precision,micro_precision,macro_precision
2,GCN,2,sigmoid,0.242974,0.355531,0.254752,0.425560,0.425560,0.439947,0.316447,0.305293,0.289438
2,TgGAT,2,sigmoid,0.466763,0.461200,0.456903,0.552042,0.552042,0.554206,0.411051,0.396030,0.394780
2,TgSAGE,2,sigmoid,0.280934,0.345625,0.260538,0.413702,0.413702,0.398803,0.233912,0.296786,0.212378
2,TgGIN,2,sigmoid,0.261864,0.312603,0.274893,0.374177,0.374177,0.425361,0.489034,0.268431,0.460854
2,GCRN,2,sigmoid,0.337920,0.362135,0.312767,0.433465,0.433465,0.403577,0.303638,0.310964,0.284289
2,DynAERNN,2,sigmoid,0.171068,0.322510,0.172461,0.386034,0.386034,0.400000,0.110674,0.276938,0.110775
2,VGRNN,2,sigmoid,0.274601,0.425977,0.213829,0.509881,0.509881,0.400000,0.188401,0.365784,0.146314
2,EvolveGCN,2,sigmoid,0.305747,0.315905,0.296218,0.378129,0.378129,0.375797,0.267965,0.271267,0.257000
2,CTGCN-C,2,sigmoid,0.325948,0.357733,0.294100,0.428195,0.428195,0.401166,0.303849,0.307183,0.279894


## DBLP 6 

In [3]:
methods = [
    'GCN', 'GAT', #'SAGE', 'GIN', 
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'GCRN', 'TIMERS', 'DynAE',# 'DynRNN',
    'DynAERNN', 'DynGEM', 'DySAT',
    'VGRNN', 'EvolveGCN', 'CTGCN-C',
]
df, model_ls = print_report(exp='dblp', methods=methods, threshold=0.5)

GCN
6766 16778 1634 3880 939 2691
GAT
6766 16778 1634 3880 939 2691
TgGCN
6766 16778 1634 3880 939 2691
TgGAT
6766 16778 1634 3880 939 2691
TgSAGE
6766 16778 1634 3880 939 2691
TgGIN
6766 16778 1634 3880 939 2691
GCRN
6766 16778 1634 3880 939 2691
TIMERS
6766 16778 1634 3880 939 2691
DynAE
5926 14534 1634 3880 939 2691
DynAERNN
5926 14534 1634 3880 939 2691
DynGEM
6766 16778 1634 3880 939 2691
DySAT
6766 16778 1634 3880 939 2691
VGRNN
6766 16778 1634 3880 939 2691
EvolveGCN
6766 16778 1634 3880 939 2691
CTGCN-C
6766 16778 1634 3880 939 2691
Full report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
0,train,0.574367,0.446195,0.564089,0.000000,0.557196,0.000000,0.573204,0.000000,0.570806,0.000000,GCN,0.574367,0.446195
1,val,0.599924,0.456618,0.516876,0.000000,0.522810,0.000000,0.587490,0.000000,0.587456,0.000000,GCN,0.599924,0.456618
2,test,0.601287,0.409750,0.522419,0.000000,0.503811,0.000000,0.607165,0.000000,0.607500,0.000000,GCN,0.607500,0.409750
3,train,0.500000,0.446453,0.551130,0.000000,0.551264,0.000000,0.568495,0.000000,0.568382,0.000000,GAT,0.568495,0.446453
4,val,0.500000,0.457191,0.559633,0.000000,0.559687,0.000000,0.571471,0.000000,0.571209,0.000000,GAT,0.571471,0.457191
5,test,0.500000,0.411031,0.536258,0.000000,0.536100,0.000000,0.574108,0.000000,0.576238,0.000000,GAT,0.576238,0.411031
6,train,0.539605,0.203712,0.538854,0.000000,0.559803,0.000000,0.528170,0.000000,0.514325,0.000000,TgGCN,0.559803,0.203712
7,val,0.552159,0.205827,0.551789,0.000000,0.569383,0.000000,0.505544,0.000000,0.515341,0.000000,TgGCN,0.569383,0.205827
8,test,0.577050,0.258741,0.577708,0.000000,0.523415,0.000000,0.513608,0.000000,0.513327,0.000000,TgGCN,0.577708,0.258741
9,train,0.528802,0.434968,0.526560,0.000000,0.550404,0.000000,0.562297,0.000000,0.561617,0.000000,TgGAT,0.562297,0.434968


Test report


,dataset,sigmoid_auc,sigmoid_f1,HAD_auc,HAD_f1,AVG_auc,AVG_f1,L1_auc,L1_f1,L2_auc,L2_f1,method,best_AUC,best_F1
2,test,0.601287,0.409750,0.522419,0.000000,0.503811,0.000000,0.607165,0.000000,0.607500,0.000000,GCN,0.607500,0.409750
5,test,0.500000,0.411031,0.536258,0.000000,0.536100,0.000000,0.574108,0.000000,0.576238,0.000000,GAT,0.576238,0.411031
8,test,0.577050,0.258741,0.577708,0.000000,0.523415,0.000000,0.513608,0.000000,0.513327,0.000000,TgGCN,0.577708,0.258741
11,test,0.538524,0.416469,0.540790,0.000000,0.515580,0.000000,0.589482,0.000000,0.590090,0.000000,TgGAT,0.590090,0.416469
14,test,0.532665,0.373384,0.534901,0.000000,0.505136,0.000000,0.573757,0.000000,0.574738,0.000000,TgSAGE,0.574738,0.373384
17,test,0.565183,0.209192,0.560961,0.000000,0.504169,0.000000,0.520415,0.000000,0.539824,0.000000,TgGIN,0.565183,0.209192
20,test,0.533966,0.402253,0.561693,0.038229,0.555413,0.000000,0.558352,0.224443,0.559792,0.000000,GCRN,0.561693,0.402253
23,test,0.680806,0.477099,0.744242,0.273128,0.500532,0.000000,0.562489,0.272548,0.566395,0.276086,TIMERS,0.744242,0.477099
26,test,0.504422,0.411031,0.524896,0.000000,0.564302,0.000000,0.538787,0.000000,0.532422,0.000000,DynAE,0.564302,0.411031
29,test,0.556475,0.411031,0.554615,0.000000,0.565709,0.000000,0.521546,0.000000,0.521284,0.000000,DynAERNN,0.565709,0.411031


Selected methods


,method,sigmoid_auc,HAD_auc,AVG_auc,L1_auc,L2_auc,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,GCN,0.601287,0.522419,0.503811,0.607165,0.607500,0.607500,0.409750,0.000000,0.000000,0.000000,0.000000,0.409750
11,TgGAT,0.538524,0.540790,0.515580,0.589482,0.590090,0.590090,0.416469,0.000000,0.000000,0.000000,0.000000,0.416469
14,TgSAGE,0.532665,0.534901,0.505136,0.573757,0.574738,0.574738,0.373384,0.000000,0.000000,0.000000,0.000000,0.373384
17,TgGIN,0.565183,0.560961,0.504169,0.520415,0.539824,0.565183,0.209192,0.000000,0.000000,0.000000,0.000000,0.209192
20,GCRN,0.533966,0.561693,0.555413,0.558352,0.559792,0.561693,0.402253,0.038229,0.000000,0.224443,0.000000,0.402253
23,TIMERS,0.680806,0.744242,0.500532,0.562489,0.566395,0.744242,0.477099,0.273128,0.000000,0.272548,0.276086,0.477099
26,DynAE,0.504422,0.524896,0.564302,0.538787,0.532422,0.564302,0.411031,0.000000,0.000000,0.000000,0.000000,0.411031
29,DynAERNN,0.556475,0.554615,0.565709,0.521546,0.521284,0.565709,0.411031,0.000000,0.000000,0.000000,0.000000,0.411031
32,DynGEM,0.574835,0.552129,0.521987,0.504054,0.523481,0.574835,0.411031,0.136090,0.006369,0.000000,0.295067,0.411031
35,DySAT,0.516052,0.523657,0.516769,0.526067,0.529218,0.529218,0.403969,0.000000,0.000000,0.008466,0.014629,0.403969


In [4]:
name = 'dblp'
data = pd.read_pickle(f'./data/{name}/0.input/data.pkl')
all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
cid2cname = pd.read_pickle('../DySubG/dataset/dblp/cid2cname.pkl')
cname2cid = {j:i for i,j in cid2cname.items() if j in all_labels}
label_mapping = {j:i for i,j in cid2cname.items() if j in all_labels}
test_data = {i: d for i,d in data.items() if d['dataset']=='test'}
ground_truth, pred_idx = get_ground_truth(test_data, all_labels, label_mapping)

In [5]:
ls = []
for method in methods:
    print(method)
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(
                    name, method, time_id), index_col=0, sep='\t').values
    models = model_ls[method]
    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    pd_res['method'] = method
    ls.append(pd_res)
results = pd.concat(ls)

GCN


,k,recall,mrr,map,ndcg,operator
1,1,0.276501,0.421488,0.276501,0.421488,sigmoid
1,1,0.166033,0.256198,0.166033,0.256198,HAD
1,1,0.201295,0.297521,0.201295,0.297521,AVG
1,1,0.275675,0.414876,0.275675,0.414876,L1
1,1,0.273609,0.418182,0.273609,0.418182,L2


GAT


,k,recall,mrr,map,ndcg,operator
1,1,0.173333,0.261157,0.173333,0.261157,sigmoid
1,1,0.118926,0.208264,0.118926,0.208264,HAD
1,1,0.118926,0.208264,0.118926,0.208264,AVG
1,1,0.173196,0.269421,0.173196,0.269421,L1
1,1,0.179532,0.287603,0.179532,0.287603,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
1,1,0.232700,0.342149,0.232700,0.342149,sigmoid
1,1,0.190275,0.289256,0.190275,0.289256,HAD
1,1,0.187796,0.287603,0.187796,0.287603,AVG
1,1,0.128375,0.196694,0.128375,0.196694,L1
1,1,0.133333,0.201653,0.133333,0.201653,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
1,1,0.175262,0.261157,0.175262,0.261157,sigmoid
1,1,0.126777,0.216529,0.126777,0.216529,HAD
1,1,0.187796,0.287603,0.187796,0.287603,AVG
1,1,0.120992,0.209917,0.120992,0.209917,L1
1,1,0.120992,0.209917,0.120992,0.209917,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
1,1,0.168788,0.266116,0.168788,0.266116,sigmoid
1,1,0.165620,0.252893,0.165620,0.252893,HAD
1,1,0.201295,0.297521,0.201295,0.297521,AVG
1,1,0.170661,0.257851,0.170661,0.257851,L1
1,1,0.153030,0.234711,0.153030,0.234711,L2


In [ ]:
0.276501	0.421488	0.276501	0.421488	
0.198402	0.307438	0.198402	0.307438	
0.187796	0.287603	0.187796	0.287603	
0.201295	0.297521	0.201295	0.297521
0.242617	0.352066	0.242617	0.352066	
0.242617	0.352066	0.242617	0.352066	
0.242617	0.352066	0.242617	0.352066	
0.216171	0.319008	0.216171	0.319008	
0.192424	0.285950	0.192424	0.285950
0.232700	0.342149	0.232700	0.342149
0.187796	0.287603	0.187796	0.287603
0.201295	0.297521	0.201295	0.297521

In [6]:
exp = 'dblp'
k = 2
time_id = 7 - 1
for method in methods:
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(
                    exp, method, time_id), index_col=0, sep='\t').values
    models = model_ls[method]
    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==k])

GCN


,k,recall,mrr,map,ndcg,operator
2,2,0.459201,0.511570,0.389642,0.464808,sigmoid
2,2,0.325868,0.356198,0.256309,0.321632,HAD
2,2,0.320220,0.386777,0.266157,0.326112,AVG
2,2,0.465675,0.511570,0.390331,0.465732,L1
2,2,0.456997,0.504959,0.387231,0.460794,L2


GAT


,k,recall,mrr,map,ndcg,operator
2,2,0.372424,0.395868,0.277314,0.349229,sigmoid
2,2,0.320771,0.340496,0.226763,0.297349,HAD
2,2,0.320220,0.342149,0.224972,0.296474,AVG
2,2,0.374904,0.393388,0.283650,0.356047,L1
2,2,0.359339,0.392562,0.279174,0.348789,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
2,2,0.417190,0.459504,0.332686,0.408686,sigmoid
2,2,0.343636,0.386777,0.275937,0.344124,HAD
2,2,0.430413,0.438843,0.316846,0.400438,AVG
2,2,0.370992,0.340496,0.262493,0.326793,L1
2,2,0.370992,0.342975,0.264972,0.328623,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
2,2,0.360854,0.391736,0.273664,0.343109,sigmoid
2,2,0.328898,0.350413,0.232824,0.307060,HAD
2,2,0.430413,0.438843,0.316846,0.400438,AVG
2,2,0.322287,0.343802,0.226625,0.300684,L1
2,2,0.322287,0.343802,0.226625,0.300684,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
2,2,0.362094,0.385124,0.273251,0.342914,sigmoid
2,2,0.331873,0.356198,0.258182,0.318189,HAD
2,2,0.329669,0.366116,0.277466,0.326073,AVG
2,2,0.305289,0.342149,0.248099,0.302222,L1
2,2,0.283113,0.323140,0.225716,0.280597,L2


In [ ]:
0.465675	0.511570	0.390331	0.465732
0.380551	0.411570	0.305069	0.373640	
0.354876	0.380165	0.277328	0.342097
0.355482	0.369421	0.258168	0.333664
0.454242	0.460331	0.365882	0.434344
0.430413	0.471074	0.344256	0.420689	
0.361543	0.440496	0.306860	0.374263
0.442948	0.440496	0.347011	0.413908
0.358595	0.395041	0.283705	0.349732
0.430413	0.438843	0.316846	0.400438	
0.430413	0.438843	0.316846	0.400438
0.362094	0.385124	0.273251	0.342914

In [7]:
exp = 'dblp'
k = 3
time_id = 7 - 1
for method in methods:
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(
                    exp, method, time_id), index_col=0, sep='\t').values
    models = model_ls[method]
    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==k])

GCN


,k,recall,mrr,map,ndcg,operator
3,3,0.620689,0.560055,0.459486,0.532226,sigmoid
3,3,0.524904,0.423967,0.335556,0.413396,HAD
3,3,0.441212,0.424793,0.318186,0.383031,AVG
3,3,0.601405,0.550689,0.452048,0.521940,L1
3,3,0.611460,0.550689,0.457443,0.526566,L2


GAT


,k,recall,mrr,map,ndcg,operator
3,3,0.516694,0.437741,0.340638,0.417233,sigmoid
3,3,0.448375,0.380165,0.281455,0.356196,HAD
3,3,0.441212,0.380165,0.277002,0.352164,AVG
3,3,0.518898,0.440220,0.342796,0.419093,L1
3,3,0.505675,0.440496,0.339330,0.413355,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
3,3,0.538953,0.497521,0.383926,0.458069,sigmoid
3,3,0.510771,0.439118,0.346791,0.419622,HAD
3,3,0.549339,0.470248,0.372456,0.450315,AVG
3,3,0.582066,0.420937,0.342466,0.431100,L1
3,3,0.563747,0.419559,0.337048,0.423428,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
3,3,0.502231,0.431956,0.334417,0.408797,sigmoid
3,3,0.449477,0.388430,0.285266,0.360589,HAD
3,3,0.558788,0.479063,0.370289,0.452425,AVG
3,3,0.441212,0.381267,0.278242,0.353200,L1
3,3,0.441212,0.381267,0.278242,0.353200,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
3,3,0.527025,0.439669,0.341212,0.420165,sigmoid
3,3,0.488264,0.410193,0.323232,0.393997,HAD
3,3,0.572287,0.436088,0.381405,0.448648,AVG
3,3,0.487025,0.406612,0.321276,0.390820,L1
3,3,0.432893,0.367218,0.290170,0.352096,L2
